In [1]:
import pandas as pd

In [21]:
packages=pd.read_csv('../../docker/csv/used/community_sources.csv', sep=";")

In [22]:
packages = packages[['package','name','debian','release_snapshot']]

In [23]:
df_1 = packages.groupby(['name']).count()[['package']].reset_index()
df_1.head(2)

,name,package
0,0track:epaxos,187
1,0track:ycsb,463


In [24]:
df_2 = packages.groupby(['name','debian','release_snapshot']).count()[['package']].rename(columns={'package':'out'}).reset_index()
df_2.head(2)

,name,debian,release_snapshot,out
0,0track:epaxos,stretch,jessie,1
1,0track:epaxos,stretch,stretch,186


In [264]:
df = df_2.merge(df_1, on='name', how='left')
df['package'] = df['out']*100/df['package']
df.drop('out', axis=1, inplace=True)

df = df.pivot(index='name', columns='release_snapshot', values='package').fillna(0).reset_index()
df = df.merge(packages[['name','debian']].drop_duplicates(), on='name', how='left')
df.drop('name', axis=1, inplace=True)


df.head(2)

,buster,jessie,stretch,debian
0,0.0,0.534759,99.465241,stretch
1,0.0,7.559395,92.440605,stretch


In [194]:
#df.loc[0,'debian'] = 'buster'
#df.head(2)

In [324]:
df['dist'] = df['debian'].map({'jessie':0,'stretch':1, 'buster':2})

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['buster','jessie','stretch']], df.dist, test_size=0.2)

In [325]:
from sklearn.tree import DecisionTreeRegressor

# Define model. Specify a number for random_state to ensure same results each run
docker_model = DecisionTreeRegressor(random_state=1)

# Fit model
docker_model.fit(X_train, y_train)
print("Making predictions")
y = docker_model.predict(X_test)

Making predictions


In [326]:
from sklearn.metrics import accuracy_score
accuracy_score(y,y_test)*100

99.89858012170384

In [327]:
j=0
for i, x in enumerate(y_test):
    if x != y[i]:
        print(x,y[i])
        j = j+1
print('error', j*100/len(y))

1 2.0
error 0.10141987829614604
